## Import libraries

In [ ]:
import os
import pandas as pd
import copy
import torch
import loss
from torch import optim
from metrics import eval_metrics, get_epoch_acc
from labelme_dataloader import LabelMe
from torchvision import transforms
from eval import eval

# Import available models, you can also explore other PyTorch models
from cracknet import cracknet
from unet import UNet, UNetResnet
from segnet import SegNet, SegResNet

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Training functions

In [ ]:
def train_oneepoch(model, class_count, criterion, eval_metric, device, my_optimizer, my_lr_scheduler, dataloader):
    model.train()
    batch_loss = 0
    batch_acc_numerator = 0
    batch_acc_denominator = 0
    epoch_lr = my_lr_scheduler.get_last_lr()[0]
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        my_optimizer.zero_grad()
        mask_pred = model(inputs)
        loss = criterion(mask_pred, labels)
        loss.backward()
        my_optimizer.step()
        batch_loss += loss
        batch_acc_numerator_tmp, batch_acc_denominator_tmp = eval_metrics(mask_pred, labels, class_count, eval_metric)
        batch_acc_numerator += batch_acc_numerator_tmp
        batch_acc_denominator += batch_acc_denominator_tmp
    my_lr_scheduler.step()
    epoch_loss = batch_loss / len(dataloader)
    epoch_acc = get_epoch_acc(batch_acc_numerator, batch_acc_denominator, eval_metric)

    return epoch_loss, epoch_acc, epoch_lr

In [ ]:
def train_main(model, class_count, criterion, eval_metric, EPOCHS, DEVICE, my_optimizer, my_lr_scheduler=None, dataloaders=None, logging=False, model_name='model.pt'):
    model.to(DEVICE)
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []
    lr = []
    best_val_acc = 0

    for epoch in range(EPOCHS):
        epoch_train_loss, epoch_train_acc, epoch_lr = train_oneepoch(model, class_count, criterion, eval_metric, DEVICE, my_optimizer, my_lr_scheduler, dataloaders['train'])
        epoch_val_loss, epoch_val_acc = eval(model, class_count, criterion, eval_metric, DEVICE, dataloaders['val'])

        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
        best_state_dict = copy.deepcopy(model.state_dict())

        if logging:
            train_loss.append(epoch_train_loss.detach().cpu().numpy().tolist())
            train_acc.append(epoch_train_acc)
            val_loss.append(epoch_val_loss.detach().cpu().numpy().tolist())
            val_acc.append(epoch_val_acc)
            lr.append(epoch_lr)
        torch.cuda.empty_cache()

        print(f'Epoch {epoch}/{EPOCHS - 1}: TrainLoss: {epoch_train_loss:.4f}, TrainAcc: {epoch_train_acc:.4f}, ValLoss: {epoch_val_loss:.4f}, ValAcc: {epoch_val_acc:.4f}')

    print('Best Acc: {:4f}'.format(best_val_acc))

    # load best model weights
    model.load_state_dict(best_state_dict)
    torch.save(model, model_name + '.pt')

    # save training details
    pd.DataFrame({'Epochs':range(EPOCHS), 'Learning Rate': lr, 'Training Loss': train_loss,
                    'Training Acc': train_acc, 'Validation Loss': val_loss,
                    'Validation Acc': val_acc}).to_csv(model_name + '.csv', index = False)

    return model

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

## Training parameters

Inside the data directory, the structure should be following:
- train
    - images
        - IL 991.png
        - IL 992.png
    - labels
        - IL 991.npy
        - IL 992.npy
    - class_names.txt
- val
    - images
        - IL 993.png
    - labels
        - IL 993.npy
    - class_names.txt

class_names.txt is to specify the label class name for the training

Example content of class_names.txt:

\_background_ <br>
fault

In [ ]:
# Name the data directory and model filename
DIR = 'data/' # Data directory
MODEL_FILENAME = 'cracknet.pt' # Model filename

# Feel free to tune these parameters for training
EPOCHS = 75
BATCH_SIZE = 6
LEARNING_RATE = 0.001
PRETRAINED = True
LOGGING = True # Record loss for each epoch
CRITERION = loss.DiceLoss() # Check loss.py for all available losses
EVAL_METRIC = 'batch_pix_accuracy' # batch_pix_accuracy or batch_intersection_union

In [ ]:
dataset = {}
dataset['train'] = LabelMe(data_folder=os.path.join(DIR,'train'), transform=data_transforms['train'],
                                img_size=(1024, 1024))
dataset['val'] = LabelMe(data_folder=os.path.join(DIR,'val'), transform=data_transforms['val'],
                                img_size=(1024, 1024))
dataloaders = {x: torch.utils.data.DataLoader(dataset[x], batch_size = BATCH_SIZE,
                                            shuffle = True, num_workers = 8,
                                            drop_last = False)
                                            for x in ['train', 'val']}
class_count = len(dataset['train'].label)

In [ ]:
# Choose a model for training, you can refer to the models that have been imported above
model = cracknet(pretrained = PRETRAINED, num_classes = class_count)

my_optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE) # Check https://pytorch.org/docs/stable/optim.html for other optimizers
my_lr_scheduler = optim.lr_scheduler.StepLR(my_optimizer, step_size=25, gamma=0.1) # Check https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate for other schedulers

## Start model training

In [ ]:
train_main(model, class_count, CRITERION, EVAL_METRIC, EPOCHS, DEVICE, my_optimizer, my_lr_scheduler, dataloaders, logging = LOGGING, model_name = MODEL_FILENAME)